# Рекуррентные нейросети

Построим простейшую нейросеть для посимвольной генерации текста

In [ ]:
import pandas as pd  # для работы с данными
import time  # для оценки времени
import torch  # для написания нейросети

## Загрузка данных

Будем работать с датасетом реплик из Симпсонов. Нам нужно извлечь предобработанные тексты и закодировать их числами

In [ ]:
df = pd.read_csv('data.csv')
df.head()

,Unnamed: 0,id,episode_id,number,raw_text,timestamp_in_ms,speaking_line,character_id,location_id,raw_character_text,raw_location_text,spoken_words,normalized_text,word_count
0,0,10368,35,29,"Lisa Simpson: Maggie, look. What's that?",235000,True,9,5.0,Lisa Simpson,Simpson Home,"Maggie, look. What's that?",maggie look whats that,4.0
1,1,10369,35,30,Lisa Simpson: Lee-mur. Lee-mur.,237000,True,9,5.0,Lisa Simpson,Simpson Home,Lee-mur. Lee-mur.,lee-mur lee-mur,2.0
2,2,10370,35,31,Lisa Simpson: Zee-boo. Zee-boo.,239000,True,9,5.0,Lisa Simpson,Simpson Home,Zee-boo. Zee-boo.,zee-boo zee-boo,2.0
3,3,10372,35,33,Lisa Simpson: I'm trying to teach Maggie that ...,245000,True,9,5.0,Lisa Simpson,Simpson Home,I'm trying to teach Maggie that nature doesn't...,im trying to teach maggie that nature doesnt e...,24.0
4,4,10374,35,35,"Lisa Simpson: It's like an ox, only it has a h...",254000,True,9,5.0,Lisa Simpson,Simpson Home,"It's like an ox, only it has a hump and a dewl...",its like an ox only it has a hump and a dewlap...,18.0


In [ ]:
phrases = df['normalized_text'].tolist()  # колонка с предобработанными текстами
phrases[:10]

['maggie look whats that',
 'lee-mur lee-mur',
 'zee-boo zee-boo',
 'im trying to teach maggie that nature doesnt end with the barnyard i want her to have all the advantages that i didnt have',
 'its like an ox only it has a hump and a dewlap hump and dew-lap hump and dew-lap',
 'you know his blood type how romantic',
 'oh yeah whats my shoe size',
 'ring',
 'yes dad',
 'ooh look maggie what is that do-dec-ah-edron dodecahedron']

In [ ]:
text = [[c for c in ph] for ph in phrases if type(ph) is str]

## Создаём массив с данными

Нужно

1. Разбить данные на токены (у нас символы)
2. Закодировать числами
3. Превратить в эмбеддинги

In [ ]:
CHARS = set('abcdefghijklmnopqrstuvwxyz ')  # все символы, которые мы хотим использовать для кодировки = наш словарь
INDEX_TO_CHAR = ['none'] + [w for w in CHARS]  # все неизвестные символы будут получать тег none
CHAR_TO_INDEX = {w: i for i, w in enumerate(INDEX_TO_CHAR)}  # словарь токен-индекс

In [ ]:
len(INDEX_TO_CHAR)

28

In [ ]:
MAX_LEN = 50  # мы хотим ограничить максимальную длину ввода
X = torch.zeros((len(text), MAX_LEN), dtype=int)  # создаём пустой вектор для текста, чтобы класть в него индексы токенов
for i in range(len(text)):  # для каждого предложения
    for j, w in enumerate(text[i]):  # для каждого токена
        if j >= MAX_LEN:
            break
        X[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])

In [ ]:
X[0:5]

tensor([[ 2,  8, 14, 14, 12,  5,  1, 23, 21, 21, 26,  1, 19, 24,  8, 13, 10,  1,
         13, 24,  8, 13,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [23,  5,  5,  0,  2, 16,  7,  1, 23,  5,  5,  0,  2, 16,  7,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 4,  5,  5,  0,  6, 21, 21,  1,  4,  5,  5,  0,  6, 21, 21,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [12,  2,  1, 13,  7, 18, 12, 20, 14,  1, 13, 21,  1, 13,  5,  8, 22, 24,
          1,  2,  8, 14, 14, 12,  5,  1, 13, 24,  8, 13,  1, 20,  8, 13, 16,  7,
          5,  1, 11, 21,  5, 10, 20, 13,  1,  5, 20, 11,  1, 19],
        [12, 13, 10,  1, 23, 12, 26,  5,  1,  8, 20,  1, 21, 27,  1, 21, 20, 23,
       

## Embedding и RNN ячейки

Каждому токену мы хотим сопоставить не просто число, но вектор. Поэтому вектор текста нам нужно умножить на матрицу эмбеддингов, которая тоже будет учиться в процессе обучения нейросети. Для создания такой матрицы нам нужен слой `nn.Embedding`

In [ ]:
X[0:5].shape

torch.Size([5, 50])

In [ ]:
embeddings = torch.nn.Embedding(len(INDEX_TO_CHAR), 28)  # размер словаря * размер вектора для кодировки каждого слова
t = embeddings(X[0:5])
t.shape

torch.Size([5, 50, 28])

In [ ]:
t.shape, X[0:5].shape

(torch.Size([5, 50, 28]), torch.Size([5, 50]))

In [ ]:
rnn = torch.nn.RNN(28, 128, batch_first=True)  # на вход - размер эмбеддинга, размер скрытого состояния и порядок размерностей
o, s = rnn(t)
# вектора для слов: батч * число токенов * размер скрытого состояния
# вектор скрытого состояния: число вектров (один) * батч * размер скрытого состояния
o.shape, s.shape

(torch.Size([5, 50, 128]), torch.Size([1, 5, 128]))

Можно применять несколько рекуррентных ячеек подряд

In [ ]:
o, s2 = rnn(t, s)
o.shape, s2.shape

(torch.Size([5, 50, 128]), torch.Size([1, 5, 128]))

## Реализация сети с RNN
3 слоя:
1. Embeding (30)
2. RNN (hidden_dim=128)
3. Полносвязный слой для предсказания буквы (28, то есть размер словаря)

In [ ]:
class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.embedding = torch.nn.Embedding(28, 30)
        self.rnn = torch.nn.RNN(30, 128)
        self.out = torch.nn.Linear(128, 28)

    def forward(self, sentences, state=None):
        x = self.embedding(sentences)
        x, s = self.rnn(x) # берём выход с последнего слоя для всех токенов, а не скрытое состояние
        return self.out(x)

In [ ]:
model = Network()

In [ ]:
criterion = torch.nn.CrossEntropyLoss()  # типичный лосс многоклассовой классификации
optimizer = torch.optim.SGD(model.parameters(), lr=.05)

Обучение:

In [ ]:
for ep in range(20):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    for i in range(int(len(X) / 100)):
        # берём батч в 100 элементов
        batch = X[i * 100:(i + 1) * 100]
        X_batch = batch[:, :-1]
        Y_batch = batch[:, 1:].flatten()

        optimizer.zero_grad()
        answers = model.forward(X_batch)
        answers = answers.view(-1, len(INDEX_TO_CHAR))
        loss = criterion(answers, Y_batch)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))

Epoch 0. Time: 3.709, Train loss: 2.114
Epoch 1. Time: 3.759, Train loss: 1.852
Epoch 2. Time: 4.396, Train loss: 1.798
Epoch 3. Time: 3.662, Train loss: 1.767
Epoch 4. Time: 3.674, Train loss: 1.746
Epoch 5. Time: 4.623, Train loss: 1.731
Epoch 6. Time: 3.593, Train loss: 1.719
Epoch 7. Time: 3.571, Train loss: 1.710
Epoch 8. Time: 4.562, Train loss: 1.703
Epoch 9. Time: 3.611, Train loss: 1.697
Epoch 10. Time: 3.521, Train loss: 1.691
Epoch 11. Time: 3.872, Train loss: 1.687
Epoch 12. Time: 4.277, Train loss: 1.683
Epoch 13. Time: 3.543, Train loss: 1.680
Epoch 14. Time: 3.512, Train loss: 1.676
Epoch 15. Time: 4.583, Train loss: 1.674
Epoch 16. Time: 3.855, Train loss: 1.671
Epoch 17. Time: 3.611, Train loss: 1.669
Epoch 18. Time: 4.587, Train loss: 1.667
Epoch 19. Time: 3.600, Train loss: 1.665


## Генерация


- Сначала отправлем в модель буквы из предложения (прогревая состояние)
- Затем берём самую вероятную букву и добавляем её в предложение
- Повторяем пока не получим none (0)

In [ ]:
CHAR_TO_INDEX['none']

0

In [ ]:
def generate_sentence(word):
    sentence = list(word)
    sentence = [CHAR_TO_INDEX.get(s, 0) for s in sentence]
    answers = model.forward(torch.tensor(sentence))
    probas, indices = answers.topk(1)
    return ''.join([INDEX_TO_CHAR[ind.item()] for ind in indices.flatten()])

In [ ]:
generate_sentence('dog')

' u '

In [ ]:
generate_sentence('It is')

'nonehtn '